# Exploring results from DC power flow

In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd

import json
import os

In [2]:
data_dir = '~/research/ream-lab/CATS-CaliforniaTestSystem'

with open('pf_solution.json') as f:
    soln = json.load(f)
    print(soln['termination_status'])

LOCALLY_SOLVED


In [3]:
generators = pd.read_csv(os.path.join(data_dir, 'GIS', 'CATS_gens.csv'))
print(generators.describe)

<bound method NDFrame.describe of       PlantCode GenID   bus                              FuelType        Pg  \
0            34    1P   745            Conventional Hydroelectric  4.033363   
1           151     1  1804                     Petroleum Liquids  0.000000   
2           151     2  1804  Natural Gas Fired Combustion Turbine  0.000000   
3           161     1  1964            Conventional Hydroelectric  0.366669   
4           161     2  1964            Conventional Hydroelectric  0.366669   
...         ...   ...   ...                                   ...       ...   
3887          0     0  8862                 Synchronous Condenser  0.000000   
3888          0     0  8863                 Synchronous Condenser  0.000000   
3889          0     0  8864                 Synchronous Condenser  0.000000   
3890          0     0  8865                 Synchronous Condenser  0.000000   
3891          0     0  8867                 Synchronous Condenser  0.000000   

      Pmax  Pmin 

In [61]:
# Load California county boundaries shapefile
# Shapefile source https://gis.data.ca.gov/datasets/8713ced9b78a4abb97dc130a691a8695
county_shapes = gpd.read_file(os.path.join(data_dir, 'GIS', 'california_county_boundaries', 'cnty19_1.shp'))
county_shapes = county_shapes.to_crs("EPSG:4326")

# Create a GeoDataFrame from the generators DataFrame
generators_geo = gpd.GeoDataFrame(
    generators,
    geometry=gpd.points_from_xy(generators['Lon'], generators['Lat'], crs="EPSG:4326")
)

# Perform spatial join to get county information for each generator
joined_data = gpd.sjoin(generators_geo, county_shapes, how="left", predicate="within")

# Create a dictionary mapping PlantCode to county name
generator_to_county = dict(zip(joined_data['PlantCode'], joined_data['COUNTY_NAM']))

# Use the resulting dictionary
# Print top 5 dictionary entries for now
gen_to_county_slice = dict(list(generator_to_county.items())[0: 5])
print(str(gen_to_county_slice))


{34: 'Placer', 151: 'Stanislaus', 161: 'Stanislaus', 162: 'Stanislaus', 180: 'Shasta'}
